<a href="https://colab.research.google.com/github/daanbee/tobigs-22nd/blob/main/%ED%88%AC%EB%B9%85%EC%8A%A4_Week5_%EA%B3%BC%EC%A0%9C2_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BERT 모델 불러오기**

In [1]:
from transformers import BertModel, BertTokenizer
import torch

# 1. BERT 토크나이저와 모델을 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [3]:
# tokenizer의 내장함수를 이용해 특수 토큰을 불러오는 함수를 작성해주세요
cls_token = tokenizer.cls_token  # [CLS] token
sep_token = tokenizer.sep_token  # [SEP] token
pad_token = tokenizer.pad_token  # [PAD] token
unk_token = tokenizer.unk_token  # [UNK] token
mask_token = tokenizer.mask_token  # [MASK] token

**1.BERT Tokenizer의 특수 토큰**

아래 코드에서 특수 토큰을 불러오는 함수를 작성하고 각 토큰이 무엇을 의미하는지 적어주세요

- [UNK] : 토크나이저가 알지 못하는 단어를 만났을 때, UNK 토큰으로 대체함.
- [SEP] : 문장 간의 구분을 나타내는 토큰
- [PAD] : 입력 시퀀스를 동일한 길이로 맞추기 위해 사용
- [CLS] : 문장의 시작을 나타내는 토큰
- [MASK] : BERT가 문장을 학습할 때 특정 단어를 에측하기 위해 사용하는 토큰

**tokenizer를 이용해 문장을 토큰화**

토큰화를 통해 문자를 숫자로 변환하여 컴퓨터가 이해할 수 있도록 한다.

In [4]:
# 입력 문장
sentence = "Hello, this is a sentence for tokenization."

# 문장 토큰화 및
inputs = tokenizer(sentence, return_tensors="pt")

# 토큰 ID를 다시 토큰으로 변환
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].tolist())  # 텐서를 리스트로 변환

print("Token IDs:", inputs['input_ids'])
print("Tokens:", tokens)

Token IDs: tensor([[  101,  7592,  1010,  2023,  2003,  1037,  6251,  2005, 19204,  3989,
          1012,   102]])
Tokens: ['[CLS]', 'hello', ',', 'this', 'is', 'a', 'sentence', 'for', 'token', '##ization', '.', '[SEP]']


**2. 토큰화**
토큰화 이후에 [CLS] 토큰과 [SEP] 토큰이 추가된 이유를 작성하세요

- [CLS] 토큰이 추가된 이유 : 문장의 대표적인 요약 정보로 사용되는 토큰이다. 이에 BERT는 이 토큰의 마지막 레이어 출력을 이용해 문장 또는 문장 쌍에 대한 결정을 내린다.
- [SEP] 토큰이 추가된 이유 : 문장의 경계를 나타내고, 문장 쌍의 구분을 위해 사용되는 토큰이다. 이는 모델이 각 문장을 독립적으로 인식하도록 돕고, 각 문장의 상호작용을 학습하는 데 중요한 역할을 한다.

**감성 분류를 위한 사전 학습된 BERT 모델을 이용해 감성분류하기**

In [5]:
import torch
from transformers import BertForSequenceClassification
from torch.nn.functional import softmax

# 감성 분류를 위한 사전 학습된 모델
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = BertForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [7]:
# sentence에 대해 BERT를 이용해 감성분류 하는 함수
def sentiment_analysis(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True, max_length=512)

    model.eval()

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    probs = torch.nn.functional.softmax(logits, dim=1)
    probs_list = probs.squeeze().tolist()

    class_label = ['매우 부정적', '부정적', '중립적', '긍정적', '매우 긍정적']

    for i, prob in enumerate(probs_list):
        print(f"{class_label[i]}일 확률: {prob:.2f}")


**3. BERT를 이용하여 각각 class에 맞는 sentence 작성해보기**

1. 매우 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
2. 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
3. 중립적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
4. 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
5. 매우 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요

In [12]:
# 매우 부정적인 문장 작성
sentence = "나는 기분이 너무 너무 안좋다"

# 감성분석 수행
sentiment_analysis(sentence)


매우 부정적일 확률: 0.17
부정적일 확률: 0.21
중립적일 확률: 0.36
긍정적일 확률: 0.17
매우 긍정적일 확률: 0.09


In [13]:
# 부정적인 문장 작성
sentence = "나는 기분이 안좋다"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.15
부정적일 확률: 0.25
중립적일 확률: 0.36
긍정적일 확률: 0.17
매우 긍정적일 확률: 0.08


In [14]:
# 중립적 문장 작성
sentence = "나는 기분이 좋진 않다"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.14
부정적일 확률: 0.22
중립적일 확률: 0.35
긍정적일 확률: 0.18
매우 긍정적일 확률: 0.10


In [15]:
# 긍정적 문장 작성
sentence = "나는 행복하다"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.15
부정적일 확률: 0.15
중립적일 확률: 0.26
긍정적일 확률: 0.23
매우 긍정적일 확률: 0.21


In [16]:
# 매우 긍정적인 문장 작성
sentence = "나는 오늘 기분이 최고다"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.13
부정적일 확률: 0.15
중립적일 확률: 0.31
긍정적일 확률: 0.23
매우 긍정적일 확률: 0.18


**4. BERT의 감성 분류 기준에 대해 생각을 적어보기**


BERT가 어떤 요소에 따라 감성 분류 기준을 나누는지에 대해 적어보세요

내가 생각하는 분류 기준 : BERT는 문맥, 어휘, 문장 구조, 상황적 맥락, 훈련 데이터의 품질 등을 종합적으로 고려하여 감성을 분류합니다.